# Tarea 2: Procesamiento de Lenguaje Natural



Integrantes:

* Sebastián Alday (Rut: 18294095-K)
* Paula Canales G. (Rut: 18845155-1)
* Álvaro Neira R. (Rut: 13757209-5)
* Matias Rodriguez U. (Rut: 18362815-1)

# Introducción

In [69]:
from sklearn.model_selection import train_test_split

# 1. Creacion y limpieza de la base de datos 

## 1.1 Elija una organización que cuenta con una lista de preguntas y respuestas

Chilexpress
https://centrodeayuda.chilexpress.cl/home

## 1.2 Genere una tabla (tablaQA.xls) con las preguntas y respuestas de la organización seleccionada.

In [5]:
!wget https://www.dropbox.com/s/9zty4fuh8boofda/tablaQA.csv

--2021-12-28 08:52:50--  https://www.dropbox.com/s/9zty4fuh8boofda/tablaQA.csv
Resolving www.dropbox.com (www.dropbox.com)... 162.125.1.18, 2620:100:6016:18::a27d:112
Connecting to www.dropbox.com (www.dropbox.com)|162.125.1.18|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: /s/raw/9zty4fuh8boofda/tablaQA.csv [following]
--2021-12-28 08:52:50--  https://www.dropbox.com/s/raw/9zty4fuh8boofda/tablaQA.csv
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://uc4bf747862db751a9276461b433.dl.dropboxusercontent.com/cd/0/inline/BcujEtlmJP91L5PS3RgJmW_S43pJZnB_c2sMuwOVWoVklFA6-G0EuSk_6R1mHLmaTPgsInHhtDAAWBb5Em-OM6XcunruMvalKFM0aeyDkeaLD9ZOCbSPVJQD6QAKm6q9cagmL-L3cl9NpANFXICyKAWM/file# [following]
--2021-12-28 08:52:50--  https://uc4bf747862db751a9276461b433.dl.dropboxusercontent.com/cd/0/inline/BcujEtlmJP91L5PS3RgJmW_S43pJZnB_c2sMuwOVWoVklFA6-G0EuSk_6R1mHLmaTPgsInHhtDAAWBb5Em-OM6XcunruM

In [6]:
import pandas as pd
qa = pd.read_csv('/content/tablaQA.csv', sep=';')
qa.tail(45)

,Pregunta,Respuesta
1,¿Cuál es el plazo para retirar un envío desde ...,El plazo para retirar un envío desde una sucur...
2,¿Puede otra persona retirar mi envío de una su...,Para que una persona que no es el destinatario...
3,¿Se puede modificar la dirección de envío?,"Sí, el cambio puede ser solicitado por el remi..."
4,¿Puedo cambiar el destino del envío desde una ...,"Sí, el cambio puede ser solicitado por el remi..."
5,¿El destinatario puede cambiar la dirección de...,"No es posible. Sin embargo, si un envío no pue..."
6,Si el destinatario no se encuentra en la direc...,"Sí, puede recibirlo cualquier persona adulta q..."
7,¿Qué pasa si el destinatario no puede recibir ...,Chilexpress realiza dos intentos de entrega en...
8,¿Cuántos intentos de entrega en el domicilio r...,Chilexpress realiza dos intentos de entrega en...
9,¿Chilexpress solicita el RUT del destinatario ...,"No es necesario para un envío nacional, pero d..."
10,¿Qué es un envío sobredimensionado?,Es toda encomienda que no cumpla el estándar e...


In [4]:
qa.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 46 entries, 0 to 45
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   Pregunta   46 non-null     object
 1   Respuesta  46 non-null     object
dtypes: object(2)
memory usage: 864.0+ bytes


## 1.3 Genere una tabla (tiposmensajes.xls) con ejemplos de las siguientes clases de mensaje: "saludo", "despedida", "nombre", "informacion"

In [20]:
!wget https://www.dropbox.com/s/wqmpjpdin6jmp3a/tiposmensajes.csv

--2021-12-28 09:09:08--  https://www.dropbox.com/s/wqmpjpdin6jmp3a/tiposmensajes.csv
Resolving www.dropbox.com (www.dropbox.com)... 162.125.1.18, 2620:100:6016:18::a27d:112
Connecting to www.dropbox.com (www.dropbox.com)|162.125.1.18|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: /s/raw/wqmpjpdin6jmp3a/tiposmensajes.csv [following]
--2021-12-28 09:09:08--  https://www.dropbox.com/s/raw/wqmpjpdin6jmp3a/tiposmensajes.csv
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://uc6a6a993e646c2b52a46ad2b7b8.dl.dropboxusercontent.com/cd/0/inline/Bcv-ZdayWiBK9WQUxSpZh_hpakzlkEW8fMtWpRuq25o8AsWlWfhBjCZ3q-QkLo9BRRc97rhfz7TVn5oXpmJCECAykAx7iXSMjT8rxLBe4yNmB4O3uuSc19xFnwNm1_Ba7zU2cuBpYIwUev3WnTRUfvyG/file# [following]
--2021-12-28 09:09:08--  https://uc6a6a993e646c2b52a46ad2b7b8.dl.dropboxusercontent.com/cd/0/inline/Bcv-ZdayWiBK9WQUxSpZh_hpakzlkEW8fMtWpRuq25o8AsWlWfhBjCZ3q-QkLo9BRRc97rhfz7T

In [21]:
messages = pd.read_csv('/content/tiposmensajes.csv', sep=',')
messages.tail(59)

,Clase,Mensaje
1,saludo,Hi
2,saludo,Buenos días
3,saludo,Buenas tardes
4,saludo,Como estas
5,despedida,Chao
6,despedida,Adiós
7,despedida,Hasta luego
8,despedida,Nos vemos
9,despedida,Bye
10,nombre,Mi nombre es


## 1.4 Genere respuestas predeterminadas (respuestasDefecto.xls) para los mensajes de tipo "saludo", "despedida", "nombre"

In [22]:
!wget https://raw.githubusercontent.com/alvaro-neira/nlp-homework2/main/respuestasDefecto.csv -O /content/respuestasDefecto.csv

--2021-12-28 09:10:27--  https://raw.githubusercontent.com/alvaro-neira/nlp-homework2/main/respuestasDefecto.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.110.133, 185.199.109.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.110.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 654 [text/plain]
Saving to: ‘/content/respuestasDefecto.csv’

/content/respuestas 100%[===================>]     654  --.-KB/s    in 0s      

2021-12-28 09:10:27 (70.0 MB/s) - ‘/content/respuestasDefecto.csv’ saved [654/654]



In [23]:
answers = pd.read_csv('/content/respuestasDefecto.csv', sep=';')
answers.tail(12)

,Clase,Mensaje
1,saludo,"Hola, cuéntame qué necesitas"
2,saludo,"Buenos días, en qué puedo ayudarte?"
3,saludo,"Buenas tardes, en qué puedo ayudarte?"
4,saludo,"Un gusto saludar, en qué puedo ayudarte?"
5,despedida,Gracias por contactarte con nosotros
6,despedida,"Adios, que esté muy bien"
7,despedida,"Hasta luego, gracias por tu tiempo"
8,despedida,"Adios, espero hayamos resuelto tu inquietud"
9,despedida,"Gracias por tu tiempo, ¡hasta luego!"
10,nombre,"Mi nombre es Eva, estoy aquí para ayudarte"


## 1.5 Describa en términos generales las tablas que construyó.



*   Tabla QA: Se construyó en base a las preguntas y respuestas del centro de ayuda de Chilexpress https://centrodeayuda.chilexpress.cl/home . Se extrajo cada pregunta y respuesta de cada categoría (Envíos Nacionales, Envíos y retiros en App, Envíos internacionales, Giros en Chile y el mundo, Atención en sucursal, Cuenta empresa) y se eliminaron las preguntas repetidas. 
*   Tipos de mensajes: se construyó con mensajes tipo de saludo, despedida y nombre. Para la información se utilizó las preguntas obtenidas en la Tabla QA.
*   Respuestas por defecto: se construyó con respuestas tipo de saludo, despedida y nombre de nuestro bot Eva



# 2. Análisis de distancia

Para el primer chatbot utilizará una métrica de su elección para responder
a cada pregunta/texto del usuario.

## 2.1 Utilice algún embeddings utilizado en el curso para codificar el texto de entrada

### 2.1.1 Word2Vec

In [24]:
import nltk
import gensim
import re
import sklearn
import numpy as np

import sklearn.linear_model
import sklearn.model_selection
from sklearn.svm import LinearSVC
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler

In [25]:
def normalizer(text): #normalizes a given string to lowercase and changes all vowels to their base form
    text = text.lower() #string lowering
    text = re.sub(r'[^A-Za-zñáéíóú]', ' ', text) #replaces every punctuation with a space
    text = re.sub('á', 'a', text) #replaces special vowels to their base forms
    text = re.sub('é', 'e', text)
    text = re.sub('í', 'i', text)
    text = re.sub('ó', 'o', text)
    text = re.sub('ú', 'u', text)
    return text

def preprocessor(text):
  text = normalizer(text)
  tokens = nltk.tokenize.casual_tokenize(text)
  #if len(tokens)==1:
  #  tokens=text
  return tokens

def vectorizer(text, model): #returns a vector representation from a list of words and a given model
    vectors = []
    for i in text:
        try:
            vectors.append(model.wv[i])
        except:
            pass
    return(np.nan_to_num(np.mean(vectors,axis=0)))

In [26]:
corpus = messages.Mensaje.tolist()
corpus_preprocessed = list(map(preprocessor,corpus))


In [29]:
model = gensim.models.word2vec.Word2Vec(sentences = corpus_preprocessed)


### 2.1.2 ELMO

In [14]:
!pip install overrides==3.1.0
!pip install elmoformanylangs

  Created wheel for overrides: filename=overrides-3.1.0-py3-none-any.whl size=10186 sha256=01f9304c6601895b0584d4a17842b9e0506c0962a463a986d50a2af766c93e22
  Stored in directory: /root/.cache/pip/wheels/3a/0d/38/01a9bc6e20dcfaf0a6a7b552d03137558ba1c38aea47644682
Successfully built overrides
     |████████████████████████████████| 42 kB 763 kB/s 


In [15]:
!wget http://vectors.nlpl.eu/repository/11/145.zip
!unzip 145.zip

--2021-12-28 08:23:08--  http://vectors.nlpl.eu/repository/11/145.zip
Resolving vectors.nlpl.eu (vectors.nlpl.eu)... 129.240.189.181
Connecting to vectors.nlpl.eu (vectors.nlpl.eu)|129.240.189.181|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 419687058 (400M) [application/zip]
Saving to: ‘145.zip’

145.zip             100%[===================>] 400.24M  10.4MB/s    in 41s     

2021-12-28 08:23:51 (9.71 MB/s) - ‘145.zip’ saved [419687058/419687058]

Archive:  145.zip
  inflating: char.dic                
  inflating: config.json             
  inflating: encoder.pkl             
  inflating: meta.json               
  inflating: README                  
  inflating: token_embedder.pkl      
  inflating: word.dic                


In [16]:
import elmoformanylangs

In [17]:
e = elmoformanylangs.Embedder('')

2021-12-28 08:24:49,019 WARNING: Could not find config.  Trying cnn_50_100_512_4096_sample.json
2021-12-28 08:24:49,020 WARNING: Could not find config.  Trying /usr/local/lib/python3.7/dist-packages/elmoformanylangs/configs/cnn_50_100_512_4096_sample.json
2021-12-28 08:24:49,114 INFO: char embedding size: 2637
2021-12-28 08:24:49,917 INFO: word embedding size: 185214
2021-12-28 08:25:01,943 INFO: Model(
  (token_embedder): ConvTokenEmbedder(
    (word_emb_layer): EmbeddingLayer(
      (embedding): Embedding(185214, 100, padding_idx=3)
    )
    (char_emb_layer): EmbeddingLayer(
      (embedding): Embedding(2637, 50, padding_idx=2634)
    )
    (convolutions): ModuleList(
      (0): Conv1d(50, 32, kernel_size=(1,), stride=(1,))
      (1): Conv1d(50, 32, kernel_size=(2,), stride=(1,))
      (2): Conv1d(50, 64, kernel_size=(3,), stride=(1,))
      (3): Conv1d(50, 128, kernel_size=(4,), stride=(1,))
      (4): Conv1d(50, 256, kernel_size=(5,), stride=(1,))
      (5): Conv1d(50, 512, kernel

In [18]:
sentences = list(map(preprocessor,messages.Mensaje))
features_raw = e.sents2elmo(sentences)

2021-12-28 08:25:06,359 INFO: 1 batches, avg len: 10.8


## 2.2 Con el texto de entrada codificado y usando embeddings proponga una manera de identificar la clase del texto de entrada.

In [27]:
def predict_class(question):
  tokens = preprocessor(question)
  vector = vectorizer(tokens,model).reshape(1, -1)
  return clf.predict(vector)

### 2.2.1 Word2Vec

In [20]:
messages

,Clase,Mensaje
0,saludo,Hola
1,saludo,Hi
2,saludo,Buenos días
3,saludo,Buenas tardes
4,saludo,Como estas
5,despedida,Chao
6,despedida,Adiós
7,despedida,Hasta luego
8,despedida,Nos vemos
9,despedida,Bye


In [32]:
features = np.zeros(shape=(len(messages),model.wv.vectors.shape[1]))
for i,msg in enumerate(messages.Mensaje):
  features[i,:] = vectorizer(preprocessor(msg),model)

cv_results = sklearn.model_selection.cross_validate(sklearn.linear_model.LogisticRegression(max_iter=10000),features,messages.Clase)
cv_results["test_score"].mean()

/usr/local/lib/python3.7/dist-packages/numpy/core/fromnumeric.py:3373: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
/usr/local/lib/python3.7/dist-packages/numpy/core/_methods.py:170: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_split.py:680: UserWarning: The least populated class in y has only 4 members, which is less than n_splits=5.
  UserWarning,


0.7666666666666667

In [72]:
features[0]

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])

In [33]:
#clf = sklearn.linear_model.LogisticRegression()
#clf.fit(features,messages.Clase)
clf = make_pipeline(StandardScaler(),LinearSVC(random_state=0, tol=1e-5))
clf.fit(features,np.array(messages.Clase))

/usr/local/lib/python3.7/dist-packages/sklearn/svm/_base.py:1208: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  ConvergenceWarning,


Pipeline(steps=[('standardscaler', StandardScaler()),
                ('linearsvc', LinearSVC(random_state=0, tol=1e-05))])

In [23]:
vectorprueba = vectorizer(preprocessor("Hola buenas"),model).reshape(1, -1)
vectorprueba

/usr/local/lib/python3.7/dist-packages/numpy/core/fromnumeric.py:3373: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
/usr/local/lib/python3.7/dist-packages/numpy/core/_methods.py:170: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


array([[0.]])

In [24]:
preprocessor("Hola buenas")

['hola', 'buenas']

In [25]:
vectorsprueba = []
for i in ['mi',"nombre","es"]:
    try:
        vectorsprueba.append(model.wv[i])

        #print(i)
        #print(vectorsprueba)
    except:
        pass
print(model.wv["mi"])
#print(vectorsprueba)
#print(np.nan_to_num(np.mean(vectorsprueba,axis=0)))

[-3.1502217e-03  2.5825892e-03 -4.0762071e-03  3.9617820e-03
  1.9476160e-03 -3.1810866e-03 -8.7596453e-04  1.9590431e-03
  3.2724806e-03  3.5598485e-03  4.5216461e-03 -4.0560324e-05
 -3.6433612e-03 -1.2935636e-03 -4.4421377e-03 -3.8912597e-03
  2.1806073e-03 -3.2107611e-03  8.4867829e-04  4.3419506e-03
  3.0582254e-03 -4.6547749e-03  4.0749880e-03 -4.9179373e-03
 -3.8248538e-03  4.6182694e-03  3.5360400e-04  1.6719312e-03
 -4.2287363e-03  2.5930307e-03 -1.6643433e-03  3.5167772e-03
 -2.0356858e-03  4.8981127e-03 -1.4330854e-03 -3.7793603e-03
  1.4277394e-03  1.2940306e-03  2.7518740e-03 -1.7304072e-03
 -2.7185790e-03  4.7277329e-03 -2.0393186e-04 -3.4719785e-03
  2.1241081e-03 -1.3393328e-04  2.0107059e-03  9.8233833e-04
  3.9504860e-03 -1.1994076e-03  8.6671091e-04 -4.2403569e-03
 -3.1138994e-03 -4.9275197e-03  2.1345669e-03  4.9056150e-03
  3.6620381e-03  2.3406763e-03 -4.2677415e-03 -4.2322809e-03
  2.2960890e-03  1.6138684e-03 -4.4024573e-03  3.9392407e-03
 -5.5619847e-04  3.33828

In [28]:
print(predict_class('¿Sólo las empresas pueden tener una cuenta con Chilexpress?'))
print(predict_class('Mi nombre es'))
# print(predict_class('hola buenas'))
print(predict_class('¿puedo retirar un giro de dinero?'))
print(predict_class('Puede otra persona retirar mi pedido'))

['información']
['nombre']
['información']
['información']


### 2.2.2 ELMO

In [29]:
features_elmo = []
for doc in features_raw:
  features_elmo.append(doc.mean(0))
features_elmo = np.vstack(features_elmo)

cv_results = sklearn.model_selection.cross_validate(sklearn.linear_model.LogisticRegression(max_iter=1000),features_elmo,messages.Clase)
cv_results["test_score"].mean()

/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_split.py:680: UserWarning: The least populated class in y has only 4 members, which is less than n_splits=5.
  UserWarning,


0.8833333333333334

In [30]:
clf = sklearn.linear_model.LogisticRegression()
clf.fit(features_elmo,messages.Clase)

LogisticRegression()

In [31]:
# print(predict_class('¿Sólo las empresas pueden tener una cuenta con Chilexpress?'))
# print(predict_class('Hola buen día como estas'))
# print(predict_class('Mi nombre es Paula'))
# print(predict_class('¿puedo retirar un giro de dinero?'))
# print(predict_class('Puede otra persona retirar mi pedido'))

ValueError: ignored

## 2.3 Si el texto de entrada es del tipo "información”, busque ahora la pregunta mas similar y retorne la respuesta asociada.

## 2.4 Reporte el resultado con textos de prueba.

# 3. Análisis transformer 

Para el segundo chatbot utilice un transformer (por ejemplo BERT).

In [64]:
!pip install transformers
from transformers import pipeline, AutoTokenizer, Trainer, TrainingArguments, AutoModel, AutoModelForSequenceClassification

In [2]:
# Transformer NER
nlp_ner = pipeline("ner", model="mrm8488/bert-spanish-cased-finetuned-ner",
    tokenizer=('mrm8488/bert-spanish-cased-finetuned-ner', {"use_fast": False})
    )
# Transformer QA
nlp_qa = pipeline('question-answering', 
        model='mrm8488/distill-bert-base-spanish-wwm-cased-finetuned-spa-squad2-es',
        tokenizer=('mrm8488/distill-bert-base-spanish-wwm-cased-finetuned-spa-squad2-es', {"use_fast": False})
    )
# Lo necesitamos para sacar el texto asociado a cada token. La predicción de ner viene con [UNK] cuando no reconoce el
# como un token
tok = AutoTokenizer.from_pretrained('mrm8488/bert-spanish-cased-finetuned-ner')

Downloading:   0%|          | 0.00/829 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/419M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/136 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/237k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/112 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/465 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/419M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/135 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/237k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/112 [00:00<?, ?B/s]

In [34]:
context_text = ""
for index, row in qa.iterrows():
    predicted = predict_class(row['Pregunta'])
    if predicted == 'información':
        context_text = context_text + row['Pregunta'] + " " + row['Respuesta']

In [35]:
context_text

'¿Qué es la Orden de Transporte (OT)? La Orden de Transporte (OT) es el número asociado al envío. Tiene de 10 a 12 números y no contiene letras. Ej. 001234567890. La Orden de Transporte aparecerá en el comprobante que te entreguen en la sucursal. También en la sección “Historial de envíos y retiros” de la App. Y si tu envío se generó en el sitio web te llegará un correo electrónico con la Orden de Transporte. Con ella, puedes hacer seguimiento a tu envío en www.chilexpress.cl en “Rastrea tu envío”, en nuestro Centro de Ayuda o llamando al 600 2000 102 opción 1.¿Cuál es el plazo para retirar un envío desde una sucursal de Chilexpress? El plazo para retirar un envío desde una sucursal es de 5 días hábiles. Una vez finalizado dicho plazo, el envío será devuelto al origen. Si el envío se generó en una oficina comercial será devuelto a la misma oficina, en cambio si se generó desde un retiro, será devuelto al domicilio.\n¿Puede otra persona retirar mi envío de una sucursal Chilexpress? Para

## 3.1 Utilice el modelo transformer para clasificar el texto de entrada, y para extraer la respuesta de la tabla de preguntas y respuestas cuando el mensaje sea del tipo “información”.

## 3.2 Reporte el tipo de red, y las métricas de entrenamiento usadas

## 3.3 Reporte el resultado con los textos de prueba.

# Conclusión

In [60]:
# Transformer Text Classification
bert_classificator = pipeline('text-classification', 
        model='dccuchile/bert-base-spanish-wwm-uncased',
        tokenizer=('dccuchile/bert-base-spanish-wwm-uncased', {"use_fast": False})
    )




loading configuration file https://huggingface.co/dccuchile/bert-base-spanish-wwm-uncased/resolve/main/config.json from cache at /root/.cache/huggingface/transformers/2416dab24674c27b5521594d6aa0929fc843a024c96711b1b5015cdff867291f.afa3630b664b4bd3e82d41660bdb96ec13236bbceadb0ae7c45c7c19f58652c7
Model config BertConfig {
  "_name_or_path": "dccuchile/bert-base-spanish-wwm-uncased",
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "output_past": true,
  "pad_token_id": 1,
  "position_embedding_type": "absolute",
  "transformers_version": "4.15.0",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 31002


In [63]:
training_args = TrainingArguments(
    output_dir="output_dir_folder",
    num_train_epochs=10,
    per_device_train_batch_size=16,  # batch size per device during training
    weight_decay=0.01,               # strength of weight decay
    load_best_model_at_end=True,
    logging_steps=200,
    evaluation_strategy="steps",
    save_steps=600, 
)

using `logging_steps` to initialize `eval_steps` to 200
PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).


In [66]:
model = AutoModel.from_pretrained('dccuchile/bert-base-spanish-wwm-uncased')


loading configuration file https://huggingface.co/dccuchile/bert-base-spanish-wwm-uncased/resolve/main/config.json from cache at /root/.cache/huggingface/transformers/2416dab24674c27b5521594d6aa0929fc843a024c96711b1b5015cdff867291f.afa3630b664b4bd3e82d41660bdb96ec13236bbceadb0ae7c45c7c19f58652c7
Model config BertConfig {
  "_name_or_path": "dccuchile/bert-base-spanish-wwm-uncased",
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "output_past": true,
  "pad_token_id": 1,
  "position_embedding_type": "absolute",
  "transformers_version": "4.15.0",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 31002


In [70]:
(train_texts,validation_texts,train_labels,validationlabels)=train_test_split(news_text, labels, test_size=0.3)

In [68]:
trainer = Trainer( model=model,args=training_args, train_dataset=messages)
trainer.train()

***** Running training *****
  Num examples = 60
  Num Epochs = 10
  Instantaneous batch size per device = 16
  Total train batch size (w. parallel, distributed & accumulation) = 16
  Gradient Accumulation steps = 1
  Total optimization steps = 40


KeyError: ignored

In [74]:
bert_classificator.fit(features,np.array(messages.Clase))

AttributeError: ignored